In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda, num_samples=config.num_train_size)
# build dimension reduction model
hnne = get_hnne_model(X, y)
# get loader
loader = get_loader(X, y, hnne=hnne)
# get val loader
val_loader = get_val_loader(robot=panda, hnne=hnne)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from /home/luca/ikpflow/weights/panda/hnne.pickle
Model load successfully from /home/luca/ikpflow/weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
mu

tensor([ 0.9267, -0.0254, -0.1134,  1.0052,  0.9972, -0.8964,  1.8243])

In [16]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0.01)
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=iflow, 
#                       num_data=1, 
#                       num_samples=1000, 
#                       inference=True)
# # ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [5]:
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=flow, 
#                       num_data=100, 
#                       num_samples=1000, 
#                       inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [6]:
nflow = get_nflow_model(flow=flow)
# df, err = test_l2_err(robot=panda, 
#                       loader=val_loader, 
#                       model=nflow, 
#                       num_data=100, 
#                       num_samples=1000, 
#                       inference=True)
# # ax1 = df.plot.scatter(x='log_prob', y='l2_err')
# df.describe()

In [7]:
total_params = sum(
	param.numel() for param in flow.parameters()
)
print(total_params)

31928526


In [18]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

mkdir /home/luca/ikpflow/data/panda/trajectory/0710013602
/home/luca/ikpflow/data/panda/trajectory/0710013602/ load successfully.


In [11]:
# generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

In [21]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=iflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[-1.20496055 -0.53399416 -0.77719017  0.07978494]
 [-1.20496055 -0.53399416 -0.77719017  0.07978494]
 [-0.59070492 -0.15764597  0.18517167 -0.45069594]]
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.002378 -42.023571      13.606661
std     0.001347   2.514380      10.724709
min     0.000543 -45.375336       1.641745
25%     0.001347 -43.609903       4.233523
50%     0.002159 -42.818270      11.861476
75%     0.002819 -40.901116      19.000035
max     0.005865 -35.047337      38.152351
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.002389 -42.843613      13.490562
std     0.001345   1.978864      10.299752
min     0.000708 -45.219070       1.077927
25%     0.001629 -43.931144       5.528476
50%     0.001937 -43.214842      11.719595
75%     0.002826 -42.470798      18.084763
max     0.005826 -38.008644      37.043166
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000  

In [22]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

config.py: Using device 'cuda:0'


In [ ]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 100
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

In [ ]:
ee_traj = load_numpy(file_path=traj_dir + 'ee_traj.npy')
traj_dir, target_poses = get_target_poses(robot=panda)
exp_dir = traj_dir + f'exp_5_single_obj/'
data = {fn: [] for fn in file_names}

# -> unrefined solutions
l2_errs = np.zeros((num_ikflow_trails,))
ang_errs = np.zeros((num_ikflow_trails,))
for ikflow_i in range(num_ikflow_trails):
    solutions, l2_err, _, _, _, _ = ik_solver.solve_n_poses(
        target_poses, 
        refine_solutions=False, 
        return_detailed=True)

    iksols = solutions.detach().cpu().numpy()
    df = qtraj_evaluation(robot=panda, qs=iksols, l2_errs=l2_err)
    l2_errs[ikflow_i], ang_errs[ikflow_i] = df.mean().values
data['ikflow_l2'] = l2_errs
data['ikflow_ang'] = ang_errs    
